In [4]:
############ LIBRARIES ###############

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
import csv
import tsfresh
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh import extract_relevant_features
from tsfresh import select_features


# Regressors
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from xgboost.sklearn import XGBRegressor 
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
import lightgbm as lgb

import warnings
from warnings import simplefilter
warnings.filterwarnings("ignore", category=RuntimeWarning)
simplefilter(action='ignore', category=FutureWarning)

# Evaluation metrics
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_percentage_error as mape

%matplotlib inline

epslon = 0.00005


def pbe(y_true, y_pred):
  if np.sum(y_true)!=0:
    return 100*(np.sum(y_pred - y_true)/np.sum(y_true))
  else:
   return 100*(np.sum(y_pred - y_true)/(np.sum(y_true)+ epslon))  

def pocid(y_true, y_pred):
    n = len(y_true)
    D = [1 if (y_pred[i] - y_pred[i-1]) * (y_true[i] - y_true[i-1]) > 0 else 0 for i in range(1, n)]
    POCID = 100 * np.sum(D) / n
    return POCID

def znorm(x):
    mean = np.mean(x)
    std_dev = np.std(x)
    # if std_dev != 0:
    x_znorm = (x - mean) / std_dev
    # else:
        # x_znorm = (x - mean) / (std_dev + np.finfo(np.float32).eps)
    return x_znorm


def get_stats_norm(series, horizon, window):
  last_subsequence = series[-(horizon+window):-horizon].values
  last_mean = np.mean(last_subsequence)
  last_std = np.std(last_subsequence)
  return last_mean, last_std

# Para predição de vendas por UF (mensal), será considerado horizon = 12
# Para predição de vendas por município (anual), será considerado horizon = 1
def train_test_split(data, horizon):
  X = data.iloc[:,:-1] # features
  y = data.iloc[:,-1] # target

  X_train = X[:-horizon] # features train
  X_test =  X[-horizon:] # features test

  y_train = y[:-horizon] # target train
  y_test = y[-horizon:] # target test
  return X_train, X_test, y_train, y_test

def recursive_multistep_forecasting(X_test, model, horizon):
  # example é composto pelas últimas observações vistas
  # na prática, é o pbeprimeiro exemplo do conjunto de teste
  example = X_test.iloc[0].values.reshape(1,-1)

  preds = []
  for i in range(horizon):
    pred = model.predict(example)[0]
    preds.append(pred)

    # Descartar o valor da primeira posição do vetor de características
    example = example[:,1:]

    # Adicionar o valor predito na última posição do vetor de características
    example = np.append(example, pred)
    example = example.reshape(1,-1)
  return preds



def rolling_window_real(series, window):
  data = []
  for i in range(len(series)-window):
    # example = znorm(np.array(series[i:i+window+1]))
    example = np.array(series[i:i+window+1])
    data.append(example)
  df = pd.DataFrame(data)
  return df


def rolling_window(series, window):
  data = []
  for i in range(len(series)-window):
    example = znorm(np.array(series[i:i+window+1]))
    data.append(example)
  df = pd.DataFrame(data)
  return df


#função para desnormatização
def znorm_reverse(x, mean_x, std_x):
  x_denormalized = (np.array(x) * std_x) + mean_x
  return x_denormalized

def get_stats_norm(series, horizon, window):
  last_subsequence = series[-(horizon+window):-horizon].values
  last_mean = np.mean(last_subsequence)
  last_std = np.std(last_subsequence)
  return last_mean, last_std




def slope(x): return (x[-1] - x[0]) / x[0] if x[0] else 0
def abs_diff_mean(x): return np.mean(np.abs(x[1:] - x[:-1])) if len(x) > 1 else 0
def diff_std(x): return np.std(x[1:] - x[:-1]) if len(x) > 1 else 0



def targeted_forecasting(X_test, model):
  # example é composto pelas últimas observações vistas
  # na prática, é o pbeprimeiro exemplo do conjunto de teste
  example = X_test.iloc[0].values.reshape(1,-1)

  preds = []
  for i in range(1):
    pred = model.predict(example)[0]
    preds.append(pred)

    # Descartar o valor da primeira posição do vetor de características
    example = example[:,1:]

    # Adicionar o valor predito na última posição do vetor de características
    example = np.append(example, pred)
    example = example.reshape(1,-1)
  return preds


In [5]:
######### DEFs ##############

import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import scipy.stats as ss
import numpy as np
import tsflex
from tsflex.features import FeatureDescriptor, FeatureCollection, FuncWrapper
import csv



def extract_estado(file_name):
    parts = file_name.split('_')
    estado = parts[1]
    return estado

def read_csv_files(folder_path):
    estados = []
    files = os.listdir(folder_path)
    for file_name in files:
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r', newline='') as csvfile:
                reader = csv.reader(csvfile)
                headers = next(reader)
                estado = extract_estado(file_name)
                estados.append(estado)
                estados.sort()
    return estados




######################################################

## 36 = 1095.75D
## 24 = 730.5D
## 12 = 365.25D

feature_descriptor_skew_TMP = FeatureDescriptor(
    function=FuncWrapper(func=ss.skew, output_names="skew"),
    series_name="TMP", 
    window="730.5D", stride="31D"
)

feature_descriptor_max_TMP = FeatureDescriptor(
    function=FuncWrapper(func=np.max, output_names="max"),
    series_name="TMP", 
    window="730.5D", stride="31D"
)

feature_descriptor_mediam_TMP = FeatureDescriptor(
    function=FuncWrapper(func=np.median, output_names="mediam"),
    series_name="TMP", 
    window="730.5D", stride="31D"
)

feature_descriptor_min_TMP = FeatureDescriptor(
    function=FuncWrapper(func=np.min, output_names="min"),
    series_name="TMP", 
    window="730.5D", stride="31D"
)

feature_descriptor_var_TMP = FeatureDescriptor(
    function=FuncWrapper(func=np.var, output_names="var"),
    series_name="TMP", 
    window="730.5D", stride="31D"
)

feature_descriptor_std_TMP = FeatureDescriptor(
    function=FuncWrapper(func=np.std, output_names="std"),
    series_name="TMP", 
    window="730.5D", stride="31D"
)

feature_descriptor_entropy_TMP = FeatureDescriptor(
    function=FuncWrapper(func=ss.entropy, output_names="entropy"),
    series_name="TMP", 
    window="730.5D", stride="31D"
)

feature_descriptor_kurtosis_TMP = FeatureDescriptor(
    function=FuncWrapper(func=ss.kurtosis, output_names="kurtosis"),
    series_name="TMP", 
    window="730.5D", stride="31D"
)

feature_descriptor_mean_TMP = FeatureDescriptor(
    function=FuncWrapper(func=np.mean, output_names="mean"),
    series_name="TMP", 
    window="730.5D", stride="31D"
)

feature_descriptor_median_abs_deviation_TMP = FeatureDescriptor(
    function=FuncWrapper(func=ss.median_abs_deviation, output_names="median_abs_deviation"),
    series_name="TMP", 
    window="730.5D", stride="31D"
)

feature_descriptor_sum_TMP = FeatureDescriptor(
    function=FuncWrapper(func=sum, output_names="sum"),
    series_name="TMP", 
    window="730.5D", stride="31D"
)

feature_descriptor_slope_TMP = FeatureDescriptor(
    function=FuncWrapper(func=slope, output_names="slope"),
    series_name="TMP", 
    window="730.5D", stride="31D"
)

feature_descriptor_abs_diff_mean_TMP = FeatureDescriptor(
    function=FuncWrapper(func=abs_diff_mean, output_names="abs_diff_mean"),
    series_name="TMP", 
    window="730.5D", stride="31D"
)

feature_descriptor_diff_std_TMP = FeatureDescriptor(
    function=FuncWrapper(func=diff_std, output_names="diff_std"),
    series_name="TMP", 
    window="730.5D", stride="31D"
)

fc = FeatureCollection(feature_descriptors=[
    feature_descriptor_skew_TMP,
    feature_descriptor_max_TMP,
    feature_descriptor_mediam_TMP,
    feature_descriptor_min_TMP,
    # feature_descriptor_var_TMP,
    feature_descriptor_std_TMP,
    # feature_descriptor_entropy_TMP,
    feature_descriptor_kurtosis_TMP,
    feature_descriptor_mean_TMP,
    feature_descriptor_median_abs_deviation_TMP,
    feature_descriptor_sum_TMP,
    feature_descriptor_slope_TMP,
    feature_descriptor_abs_diff_mean_TMP,
    feature_descriptor_diff_std_TMP
])





####################################################################


def recursive_multistep_forecasting_TsFlexN(series, model, horizon, window):

    serieatu=series[:-horizon]
    seriec = serieatu.tail(2+window).reset_index(drop=True)
    mean_norm, std_norm = get_stats_norm(series['m3'], horizon, window)
    predsreal = []

    for i in range(horizon):
        df2 = seriec
        try:
            df2['timestamp'] = pd.to_datetime(df2['timestamp'], format='%Y%m')
        except Exception as e:
            print("Error parsing 'timestamp' column:", e)
            raise
        if 'm3' not in df2.columns:
            raise ValueError("'m3' column not found in the CSV file. Columns available: {}".format(df.columns))
        df2 = df2.set_index('timestamp')
        df2 = df2.rename(columns={'m3': 'TMP'})
        example = np.array(seriec)
        new_elements = fc.calculate(data=df2, return_df=True)
        exemple_feature = znorm(new_elements.values)
        exemple_feature_df = pd.DataFrame(exemple_feature)
        exemple_feature_df.fillna(0, inplace=True)
        predn = model.predict(exemple_feature_df)
        pred = znorm_reverse(predn, mean_norm, std_norm)
        predsreal.append(pred)
        series2 = seriec.drop(seriec.index[0])

        last_timestamp = pd.to_datetime(series2['timestamp'].iloc[-1])

        year = last_timestamp.year
        month = last_timestamp.month

        if month == 12:
            next_year = year + 1
            next_month = 1
        else:
            next_year = year
            next_month = month + 1

        next_timestamp = pd.Timestamp(year=next_year, month=next_month, day=1)
        new_row = pd.DataFrame({'timestamp': [next_timestamp], 'm3': pred})
        seriec = pd.concat([series2, new_row], ignore_index=True)

    return predsreal


In [3]:
######### TsFlex SAVE #################

import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import scipy.stats as ss
import numpy as np
import tsflex
from tsflex.features import FeatureDescriptor, FeatureCollection, FuncWrapper


products = sorted([name for name in os.listdir('./uf/') if os.path.isdir(os.path.join('./uf/', name))])

horizon = 12
window = 12

for product in products:
    folder_path = f'./uf/{product}/'
    # Read the CSV files and extract estado names
    estados = read_csv_files(folder_path)
    
    for estado in estados:
    
        df = pd.read_csv(f"./uf/{product}/mensal_{estado}_{product}.csv", header=0, sep=";")

        series = df

        ###############################################################

        try:
            df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y%m')
        except Exception as e:
            print("Error parsing 'timestamp' column:", e)
            raise

        if 'm3' not in df.columns:
            raise ValueError("'m3' column not found in the CSV file. Columns available: {}".format(df.columns))

        df = df.set_index('timestamp')

        df = df.rename(columns={'m3': 'TMP'})


        ## 36 = 1095.75D

        ## 12 = 365.25

        feature_descriptor_skew_TMP = FeatureDescriptor(
            function=FuncWrapper(func=ss.skew, output_names="skew"),
            series_name="TMP", 
            window="365.25D", stride="31D"
        )

        feature_descriptor_max_TMP = FeatureDescriptor(
            function=FuncWrapper(func=np.max, output_names="max"),
            series_name="TMP", 
            window="365.25D", stride="31D"
        )

        feature_descriptor_mediam_TMP = FeatureDescriptor(
            function=FuncWrapper(func=np.median, output_names="mediam"),
            series_name="TMP", 
            window="365.25D", stride="31D"
        )

        feature_descriptor_min_TMP = FeatureDescriptor(
            function=FuncWrapper(func=np.min, output_names="min"),
            series_name="TMP", 
            window="365.25D", stride="31D"
        )

        feature_descriptor_var_TMP = FeatureDescriptor(
            function=FuncWrapper(func=np.var, output_names="var"),
            series_name="TMP", 
            window="365.25D", stride="31D"
        )

        feature_descriptor_std_TMP = FeatureDescriptor(
            function=FuncWrapper(func=np.std, output_names="std"),
            series_name="TMP", 
            window="365.25D", stride="31D"
        )

        feature_descriptor_entropy_TMP = FeatureDescriptor(
            function=FuncWrapper(func=ss.entropy, output_names="entropy"),
            series_name="TMP", 
            window="365.25D", stride="31D"
        )

        feature_descriptor_kurtosis_TMP = FeatureDescriptor(
            function=FuncWrapper(func=ss.kurtosis, output_names="kurtosis"),
            series_name="TMP", 
            window="365.25D", stride="31D"
        )

        feature_descriptor_mean_TMP = FeatureDescriptor(
            function=FuncWrapper(func=np.mean, output_names="mean"),
            series_name="TMP", 
            window="365.25D", stride="31D"
        )

        feature_descriptor_median_abs_deviation_TMP = FeatureDescriptor(
            function=FuncWrapper(func=ss.median_abs_deviation, output_names="median_abs_deviation"),
            series_name="TMP", 
            window="365.25D", stride="31D"
        )

        feature_descriptor_sum_TMP = FeatureDescriptor(
            function=FuncWrapper(func=sum, output_names="sum"),
            series_name="TMP", 
            window="365.25D", stride="31D"
        )

        feature_descriptor_slope_TMP = FeatureDescriptor(
            function=FuncWrapper(func=slope, output_names="slope"),
            series_name="TMP", 
            window="365.25D", stride="31D"
        )

        feature_descriptor_abs_diff_mean_TMP = FeatureDescriptor(
            function=FuncWrapper(func=abs_diff_mean, output_names="abs_diff_mean"),
            series_name="TMP", 
            window="365.25D", stride="31D"
        )

        feature_descriptor_diff_std_TMP = FeatureDescriptor(
            function=FuncWrapper(func=diff_std, output_names="diff_std"),
            series_name="TMP", 
            window="365.25D", stride="31D"
        )

        fc = FeatureCollection(feature_descriptors=[
            feature_descriptor_skew_TMP,
            feature_descriptor_max_TMP,
            feature_descriptor_mediam_TMP,
            feature_descriptor_min_TMP,
            # feature_descriptor_var_TMP,
            feature_descriptor_std_TMP,
            # feature_descriptor_entropy_TMP,
            feature_descriptor_kurtosis_TMP,
            feature_descriptor_mean_TMP,
            feature_descriptor_median_abs_deviation_TMP,
            feature_descriptor_sum_TMP,
            feature_descriptor_slope_TMP,
            feature_descriptor_abs_diff_mean_TMP,
            feature_descriptor_diff_std_TMP
        ])

        result_df = fc.calculate(data=df, return_df=True)

        result_df.reset_index(inplace=True)

        folder_name = 'TsFlex'
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)
        
        result_df.to_csv(f'{folder_name}/FEAT_TsFresh_{product}_{window}_{estado}.csv', index=False)

KeyboardInterrupt: 

In [3]:
######### TsFlex TEST #################

import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import scipy.stats as ss
import numpy as np
import tsflex
from tsflex.features import FeatureDescriptor, FeatureCollection, FuncWrapper

horizon = 12
window = 24


product = 'etanolhidratado'
estado = 'ac'


# products = sorted([name for name in os.listdir('./uf/') if os.path.isdir(os.path.join('./uf/', name))])
# for product in products:
#     folder_path = f'./uf/{product}/'
#     # Read the CSV files and extract estado names
#     estados = read_csv_files(folder_path)
    
#     for estado in estados:

df = pd.read_csv(f"./uf/{product}/mensal_{estado}_{product}.csv", header=0, sep=";")

series = df

###############################################################

try:
    df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y%m')
except Exception as e:
    print("Error parsing 'timestamp' column:", e)
    raise

if 'm3' not in df.columns:
    raise ValueError("'m3' column not found in the CSV file. Columns available: {}".format(df.columns))

df = df.set_index('timestamp')
df = df.rename(columns={'m3': 'TMP'})





result_df = fc.calculate(data=df, return_df=True)
result_df.reset_index(inplace=True)

result_df_timestamp = result_df[result_df.columns[0]]
result_df = result_df.drop(result_df.columns[0], axis=1)
result_df_norm = result_df.apply(znorm, axis=1)

y_norm = rolling_window(series['m3'],window)

result_df_norm['y'] = y_norm[window]

X_train, X_test, y_train, y_test = train_test_split(result_df_norm, horizon)



############## Regressores ##################

regr1 = LinearRegression()
regr2 = KNeighborsRegressor(n_neighbors = 3)
regr3 = XGBRegressor()
regr4 = SVR(kernel='rbf')
regr5 = RandomForestRegressor()
regr6 = MLPRegressor(random_state=1, activation='relu', max_iter=500)


X_train.columns = range(0, len(X_train.columns))

regr1.fit(X_train, y_train)
predictions1a = recursive_multistep_forecasting_TsFlexN(series, regr1, horizon, window)
predictions1a = [a.flatten()[0] for a in predictions1a]

regr2.fit(X_train, y_train)
predictions2a = recursive_multistep_forecasting_TsFlexN(series, regr2, horizon, window)
predictions2a = [a.flatten()[0] for a in predictions2a]

regr3.fit(X_train, y_train)
predictions3a = recursive_multistep_forecasting_TsFlexN(series, regr3, horizon, window)
predictions3a = [a.flatten()[0] for a in predictions3a]


regr4.fit(X_train, y_train)
predictions4a = recursive_multistep_forecasting_TsFlexN(series, regr4, horizon, window)
predictions4a = [a.flatten()[0] for a in predictions4a]

regr5.fit(X_train, y_train)
predictions5a = recursive_multistep_forecasting_TsFlexN(series, regr5, horizon, window)
predictions5a = [a.flatten()[0] for a in predictions5a]

regr6.fit(X_train, y_train)
predictions6a = recursive_multistep_forecasting_TsFlexN(series, regr6, horizon, window)
predictions6a = [a.flatten()[0] for a in predictions6a]


Valores_Reais = series['m3'].tail(horizon)
Valores_Reais = Valores_Reais.reset_index(drop=True)

mape_result1 = mape(Valores_Reais, predictions1a)
mape_result2 = mape(Valores_Reais, predictions2a)
mape_result3 = mape(Valores_Reais, predictions3a)
mape_result4 = mape(Valores_Reais, predictions4a)
mape_result5 = mape(Valores_Reais, predictions5a)
mape_result6 = mape(Valores_Reais, predictions6a)

pbe_result1 = pbe(Valores_Reais, predictions1a)
pbe_result2 = pbe(Valores_Reais, predictions2a)
pbe_result3 = pbe(Valores_Reais, predictions3a)
pbe_result4 = pbe(Valores_Reais, predictions4a)
pbe_result5 = pbe(Valores_Reais, predictions5a)
pbe_result6 = pbe(Valores_Reais, predictions6a)

pocid_result1 = pocid(Valores_Reais, predictions1a)
pocid_result2 = pocid(Valores_Reais, predictions2a)
pocid_result3 = pocid(Valores_Reais, predictions3a)
pocid_result4 = pocid(Valores_Reais, predictions4a)
pocid_result5 = pocid(Valores_Reais, predictions5a)
pocid_result6 = pocid(Valores_Reais, predictions6a)

predictions1b = ', '.join(f"{item:f}" for item in predictions1a)
predictions2b = ', '.join(f"{item:f}" for item in predictions2a)
predictions3b = ', '.join(f"{item:f}" for item in predictions3a)
predictions4b = ', '.join(f"{item:f}" for item in predictions4a)
predictions5b = ', '.join(f"{item:f}" for item in predictions5a)
predictions6b = ', '.join(f"{item:f}" for item in predictions6a)
    
rows_data = [
    [product,estado,'LR',mape_result1,pocid_result1,pbe_result1,predictions1b],
    [product,estado,'kNN',mape_result2,pocid_result2,pbe_result2,predictions2b],
    [product,estado,'XGB',mape_result3,pocid_result3,pbe_result3,predictions3b],
    [product,estado,'SVR',mape_result4,pocid_result4,pbe_result4,predictions4b],
    [product,estado,'RF',mape_result5,pocid_result5,pbe_result5,predictions5b],
    [product,estado,'MLP',mape_result6,pocid_result6,pbe_result6,predictions6b],          
]

# # Salva Modelos
# folder_path = f"./00-MODELS_UF_MENSAL-TsFel/"
# os.makedirs(folder_path, exist_ok=True)

# with open(os.path.join(folder_path, f'TsFel_LR_{product}_{estado}_{window}_model.pkl'), 'wb') as fd1:
#     pickle.dump(regr2, fd1)
# with open(os.path.join(folder_path, f'TsFel_KNN_{product}_{estado}_{window}_model.pkl'), 'wb') as fd2:
#     pickle.dump(regr2, fd2)
# with open(os.path.join(folder_path, f'TsFel_XGB_{product}_{estado}_{window}_model.pkl'), 'wb') as fd3:
#     pickle.dump(regr3, fd3)
# with open(os.path.join(folder_path, f'TsFel_SVR_{product}_{estado}_{window}_model.pkl'), 'wb') as fd4:
#     pickle.dump(regr4, fd4)
# with open(os.path.join(folder_path, f'TsFel_RF_{product}_{estado}_{window}_model.pkl'), 'wb') as fd5:
#     pickle.dump(regr5, fd5)
# with open(os.path.join(folder_path, f'TsFel_MLP_{product}_{estado}_{window}_model.pkl'), 'wb') as fd6:
#     pickle.dump(regr2, fd6)

# CSV Output VALORES REAIS
with open(f'TsFlex_{window}_Norm_output.csv', 'a', newline='') as file:
    writer = csv.writer(file)
    for row_data in rows_data:
        writer.writerow(row_data)



In [6]:
######### TsFlex AUTO NORM #################

import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import scipy.stats as ss
import numpy as np
import tsflex
from tsflex.features import FeatureDescriptor, FeatureCollection, FuncWrapper

horizon = 12
window = 24


# product = 'etanolhidratado'
# estado = 'ac'


products = sorted([name for name in os.listdir('./uf/') if os.path.isdir(os.path.join('./uf/', name))])
for product in products:
    folder_path = f'./uf/{product}/'
    # Read the CSV files and extract estado names
    estados = read_csv_files(folder_path)
    
    for estado in estados:

        df = pd.read_csv(f"./uf/{product}/mensal_{estado}_{product}.csv", header=0, sep=";")

        series = df

        ###############################################################

        try:
            df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y%m')
        except Exception as e:
            print("Error parsing 'timestamp' column:", e)
            raise

        if 'm3' not in df.columns:
            raise ValueError("'m3' column not found in the CSV file. Columns available: {}".format(df.columns))

        df = df.set_index('timestamp')
        df = df.rename(columns={'m3': 'TMP'})





        result_df = fc.calculate(data=df, return_df=True)
        result_df.reset_index(inplace=True)

        result_df_timestamp = result_df[result_df.columns[0]]
        result_df = result_df.drop(result_df.columns[0], axis=1)
        result_df_norm = result_df.apply(znorm, axis=1)
        

        y_norm = rolling_window(series['m3'],window)

        result_df_norm['y'] = y_norm[window]
        result_df_norm.fillna(0, inplace=True)

        
        X_train, X_test, y_train, y_test = train_test_split(result_df_norm, horizon)



        ############## Regressores ##################

        regr1 = LinearRegression()
        regr2 = KNeighborsRegressor(n_neighbors = 3)
        regr3 = XGBRegressor()
        regr4 = SVR(kernel='rbf')
        regr5 = RandomForestRegressor()
        regr6 = MLPRegressor(random_state=1, activation='relu', max_iter=500)


        X_train.columns = range(0, len(X_train.columns))

        regr1.fit(X_train, y_train)
        predictions1a = recursive_multistep_forecasting_TsFlexN(series, regr1, horizon, window)
        predictions1a = [a.flatten()[0] for a in predictions1a]

        regr2.fit(X_train, y_train)
        predictions2a = recursive_multistep_forecasting_TsFlexN(series, regr2, horizon, window)
        predictions2a = [a.flatten()[0] for a in predictions2a]

        regr3.fit(X_train, y_train)
        predictions3a = recursive_multistep_forecasting_TsFlexN(series, regr3, horizon, window)
        predictions3a = [a.flatten()[0] for a in predictions3a]


        regr4.fit(X_train, y_train)
        predictions4a = recursive_multistep_forecasting_TsFlexN(series, regr4, horizon, window)
        predictions4a = [a.flatten()[0] for a in predictions4a]

        regr5.fit(X_train, y_train)
        predictions5a = recursive_multistep_forecasting_TsFlexN(series, regr5, horizon, window)
        predictions5a = [a.flatten()[0] for a in predictions5a]

        regr6.fit(X_train, y_train)
        predictions6a = recursive_multistep_forecasting_TsFlexN(series, regr6, horizon, window)
        predictions6a = [a.flatten()[0] for a in predictions6a]


        Valores_Reais = series['m3'].tail(horizon)
        Valores_Reais = Valores_Reais.reset_index(drop=True)

        mape_result1 = mape(Valores_Reais, predictions1a)
        mape_result2 = mape(Valores_Reais, predictions2a)
        mape_result3 = mape(Valores_Reais, predictions3a)
        mape_result4 = mape(Valores_Reais, predictions4a)
        mape_result5 = mape(Valores_Reais, predictions5a)
        mape_result6 = mape(Valores_Reais, predictions6a)

        pbe_result1 = pbe(Valores_Reais, predictions1a)
        pbe_result2 = pbe(Valores_Reais, predictions2a)
        pbe_result3 = pbe(Valores_Reais, predictions3a)
        pbe_result4 = pbe(Valores_Reais, predictions4a)
        pbe_result5 = pbe(Valores_Reais, predictions5a)
        pbe_result6 = pbe(Valores_Reais, predictions6a)

        pocid_result1 = pocid(Valores_Reais, predictions1a)
        pocid_result2 = pocid(Valores_Reais, predictions2a)
        pocid_result3 = pocid(Valores_Reais, predictions3a)
        pocid_result4 = pocid(Valores_Reais, predictions4a)
        pocid_result5 = pocid(Valores_Reais, predictions5a)
        pocid_result6 = pocid(Valores_Reais, predictions6a)

        predictions1b = ', '.join(f"{item:f}" for item in predictions1a)
        predictions2b = ', '.join(f"{item:f}" for item in predictions2a)
        predictions3b = ', '.join(f"{item:f}" for item in predictions3a)
        predictions4b = ', '.join(f"{item:f}" for item in predictions4a)
        predictions5b = ', '.join(f"{item:f}" for item in predictions5a)
        predictions6b = ', '.join(f"{item:f}" for item in predictions6a)
            
        rows_data = [
            [product,estado,'LR',mape_result1,pocid_result1,pbe_result1,predictions1b],
            [product,estado,'kNN',mape_result2,pocid_result2,pbe_result2,predictions2b],
            [product,estado,'XGB',mape_result3,pocid_result3,pbe_result3,predictions3b],
            [product,estado,'SVR',mape_result4,pocid_result4,pbe_result4,predictions4b],
            [product,estado,'RF',mape_result5,pocid_result5,pbe_result5,predictions5b],
            [product,estado,'MLP',mape_result6,pocid_result6,pbe_result6,predictions6b],          
        ]

        # # Salva Modelos
        # folder_path = f"./00-MODELS_UF_MENSAL-TsFel/"
        # os.makedirs(folder_path, exist_ok=True)

        # with open(os.path.join(folder_path, f'TsFel_LR_{product}_{estado}_{window}_model.pkl'), 'wb') as fd1:
        #     pickle.dump(regr2, fd1)
        # with open(os.path.join(folder_path, f'TsFel_KNN_{product}_{estado}_{window}_model.pkl'), 'wb') as fd2:
        #     pickle.dump(regr2, fd2)
        # with open(os.path.join(folder_path, f'TsFel_XGB_{product}_{estado}_{window}_model.pkl'), 'wb') as fd3:
        #     pickle.dump(regr3, fd3)
        # with open(os.path.join(folder_path, f'TsFel_SVR_{product}_{estado}_{window}_model.pkl'), 'wb') as fd4:
        #     pickle.dump(regr4, fd4)
        # with open(os.path.join(folder_path, f'TsFel_RF_{product}_{estado}_{window}_model.pkl'), 'wb') as fd5:
        #     pickle.dump(regr5, fd5)
        # with open(os.path.join(folder_path, f'TsFel_MLP_{product}_{estado}_{window}_model.pkl'), 'wb') as fd6:
        #     pickle.dump(regr2, fd6)

        # CSV Output VALORES REAIS
        with open(f'TsFlex_{window}_Norm_output.csv', 'a', newline='') as file:
            writer = csv.writer(file)
            for row_data in rows_data:
                writer.writerow(row_data)

